<a href="https://colab.research.google.com/github/bhargav998/playing-with-data-sets/blob/main/pyspark_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=42f6365cc21918393b2a5e481a19582f5e017746d03f6cb7d3c7f9b0f5b9e063
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


Input :
Mail,mob
Renuka1992@gmail.com,9856765434 
anbu.arasu@gmail.com,9844567788 

Output:
Mail,mob
R********2@gmail.com,98****34 
a********u@gmail.com,98****88

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lit

spark = SparkSession.builder.getOrCreate()

# Masking function
def mask_email(email):
  return(email[0]+"**********"+email[8:])

def mask_mobile(mobile):
  return(mobile[0:2]+"**********"+mobile[-3:])


# Create DataFrame
df = spark.createDataFrame([("RIL",""), ("anbu.arasu@gmail.com","9844567788")], ["email", "mobile"])

# Mask the email column
masked_df = df.withColumn("email", udf(mask_email)(df.email)).withColumn("mobile", udf(mask_mobile)(df.mobile))
masked_df.show()


+--------------------+---------------+
|               email|         mobile|
+--------------------+---------------+
|R**********92@gma...|98**********434|
|a**********su@gma...|98**********788|
+--------------------+---------------+



Replace checking with cash in column

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lit,col,explode,regexp_replace,when,sum

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('/content/personal_transactions.csv',header='true')
df1=df.withColumn("Card_type",regexp_replace(df.Card_type, "Checking", "cash"))
df1.show()
df2=df1.withColumn("final amt",when(col("Transaction type")=="debit",-1*col("amount")).otherwise(col("amount")))
df2.show()
grouped_df = df2.groupBy("Customer_No").agg(sum("final amt"))
grouped_df.show()






AnalysisException: ignored

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lit,col,explode,regexp_replace,when,sum

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('/content/personal_transactions.csv',header='true')
df1=df.withColumn("Card_type",regexp_replace(df.Card_type, "Checking", "cash"))
pivoted_df = df1.groupBy("Customer_No").pivot("Transaction type").agg(sum("amount"))
c=pivoted_df.withColumn("final amt",col("credit")-col("debit")).drop("credit","debit")
c.show()


+-----------+------------------+
|Customer_No|         final amt|
+-----------+------------------+
|    1001863|           2069.64|
|    1001368|2155.7200000000003|
|    1000210|1705.1399999999999|
|    1000531|1657.1499999999996|
|    1000654| 798.1199999999999|
|    1002324|           1601.49|
|    1000501|           2720.32|
+-----------+------------------+



Maximum Amount can be debited on latest date


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lit,col,explode,regexp_replace,when,sum

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('/content/personal_transactions.csv',header='true')


AttributeError: ignored

RIL,[1000,1005,1090,1200,1000,900,890]
HDFC,[890,940,810,730,735,960,980]
INFY,[1001,902,1000,990,1230,1100,1200]

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,udf
from pyspark.sql.types import StructType, StructField, IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("Array Column Operations").getOrCreate()

def find_max_profit(pridicted):
    max_profit = 0
    buy_price = 0
    sell_price = 0

    for i in range(len(pridicted)):
        for j in range(i + 1, len(pridicted)):
            profit = pridicted[j] - pridicted[i]
            if profit > max_profit:
                max_profit = profit
                buy_price = pridicted[i]
                sell_price = pridicted[j]

    return buy_price, sell_price,max_profit


# Define the schema for the struct
schema = StructType([
    StructField("buy price", IntegerType(), nullable=False),
    StructField("sell price", IntegerType(), nullable=False),
    StructField("max profit", IntegerType(), nullable=False)
])

# Apply the UDF to the "Pridict" column and create two new columns
df_with_processed_prediction = df.withColumn("predictoutputs", udf(find_max_profit,schema)(col("pridicted")))

# Extract the struct elements into separate columns
df_with_processed_prediction = df_with_processed_prediction.select(
    col("stockid"),
    col("predictoutputs.buy price").alias("buy price"),
    col("predictoutputs.sell price").alias("sell price"),
    col("predictoutputs.max profit").alias("max profit")
)

# Sample data
data = [("RIL",[1000,1005,1090,1200,1000,900,890] ), ("HDFC",[890,940,810,730,735,960,980]), ("INFY", [1001,902,1000,990,1230,1100,1200])]

# Create a DataFrame
df = spark.createDataFrame(data, ["stockid", "pridicted"])


df_with_processed_prediction.show()



+-------+---------+----------+----------+
|stockid|buy price|sell price|max profit|
+-------+---------+----------+----------+
|    RIL|     1000|      1200|       200|
|   HDFC|      730|       980|       250|
|   INFY|      902|      1230|       328|
+-------+---------+----------+----------+



nput
RollNo, name,tamil,eng,math,sci,social 
203040, Rajesh, 10, 20, 30, 40, 50 

Output:
RollNo,name,Tamil,eng,math,sci,social ,tot
203040, Rajesh, 10, 20, 30, 40, 50,150


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,udf

data=[(203040,"Rajesh",10,20,30,40,50)]
coloums=["RollNo","name","tamil","eng","math","sci","social"]
df=spark.createDataFrame(data,coloums)
df.withColumn("total marks",col("tamil")+col("eng")+col("math")+col("sci")+col("social")).show()

+------+------+-----+---+----+---+------+-----------+
|RollNo|  name|tamil|eng|math|sci|social|total marks|
+------+------+-----+---+----+---+------+-----------+
|203040|Rajesh|   10| 20|  30| 40|    50|        150|
+------+------+-----+---+----+---+------+-----------+



Replace Checking with Cash

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('/content/personal_transactions.csv',header='true')
na_replace_df=df.na.replace("checking","cash")
na_replace_df.show()

ModuleNotFoundError: ignored

How to Remove First N lines from Header Using PySpark 

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('/content/duplicate.csv',header=True)
df.createOrReplaceTempView("sys")
res=spark.sql("select * from sys where group by Name")
res.show()


AnalysisException: ignored

AttributeError: ignored

In [ ]:
list1=["India","Australia","England","New Zealand"]

for i in range(len(list1)-1):
  for j in range(i+1,len(list1)):
    print(str(list1[i])+" "+"Vs"+" "+str(list1[j]))



India Vs Australia
India Vs England
India Vs New Zealand
Australia Vs England
Australia Vs New Zealand
England Vs New Zealand


In [ ]:
transactions = [
    {'buyer': 'Alice', 'seller': 'Bob', 'amount': 1000000},
    {'buyer': 'Bob', 'seller': 'Charlie', 'amount': 500000},
    {'buyer': 'Charlie', 'seller': 'Alice', 'amount': 750000},
    {'buyer': 'David', 'seller': 'Alice', 'amount': 100000},
    {'buyer': 'Alice', 'seller': 'Charlie', 'amount': 250000},
    {'buyer': 'Charlie', 'seller': 'David', 'amount': 300000},
    {'buyer': 'David', 'seller': 'Bob', 'amount': 150000},
    {'buyer': 'Bob', 'seller': 'David', 'amount': 200000}
]
a=[]

sorted_transactions = sorted(transactions, key=lambda x: x['amount'], reverse=True)[-1]
a.append(sorted_transactions['buyer'])
a.append(sorted_transactions['seller'])

print(a)

# Print the sorted transactions



['David', 'Alice']


Brodcast Variable (spark tuning)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

def system1(col):
  return brod.value[col]


spark = SparkSession.builder.getOrCreate()
df = spark.read.options(delimiter="|").csv('/content/drive/MyDrive/Datasets/uspopulation.csv',header=True)
df.show(5)

lookup=dict({"NY":"New york","CA":"Califonia","IL":"iRELAND","TX":"Texus","AZ":"Azuland","OH":"Ohio","CO":"Colocoldio"})
brod=spark.sparkContext.broadcast(lookup)
df1=df.withColumn("State",udf(system1)("state_code")).show()


+---------+-----------+----------+-------------+-----------+------------------+
|2019_rank|       City|State_Code|2019_estimate|2010_Census|            Change|
+---------+-----------+----------+-------------+-----------+------------------+
|        1|New York[d]|        NY|      8336817|    8175133|            0.0198|
|        2|Los Angeles|        CA|      3979576|    3792621|            0.0493|
|        3|    Chicago|        IL|      2693976|    2695598|−0.06%            |
|        4| Houston[3]|        TX|      2320268|    2100263|            0.1048|
|        5|    Phoenix|        AZ|      1680992|    1445632|            0.1628|
+---------+-----------+----------+-------------+-----------+------------------+
only showing top 5 rows

+---------+----------------+----------+-------------+-----------+------------------+----------+
|2019_rank|            City|State_Code|2019_estimate|2010_Census|            Change|     State|
+---------+----------------+----------+-------------+----------

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StructType, StructField, StringType

def convert(HIREDATE):
  year=HIREDATE[6:]
  month=HIREDATE[3:5]
  date=HIREDATE[0:2]

  return str(year),str(month),str(date)

# Define the schema for the struct
schema = StructType([
    StructField("year", StringType(), nullable=False),
    StructField("month", StringType(), nullable=False),
    StructField("date", StringType(), nullable=False)
])

df1 = df.withColumn("datesep", udf(convert,schema)(col("HIREDATE")))


df1 = df1.select(
    "*",
    col("datesep.year").alias("year"),
    col("datesep.month").alias("month"),
    col("datesep.date").alias("date"),
    )
df1=df1.drop("datesep")

spark = SparkSession.builder.getOrCreate()
df=spark.read.options(Header=True).csv("/content/drive/MyDrive/Datasets/emp.csv")
df.show()
df1.show()

+-----+------+---------+----+----------+----+----+------+------------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+---------+----+----------+----+----+------+------------+
| 7369| SMITH|    CLERK|7902|17-12-1980| 800|null|    20|  2022-01-01|
| 7499| ALLEN| SALESMAN|7698|20-02-1981|1600| 300|    30|  2022-01-01|
| 7521|  WARD| SALESMAN|7698|22-02-1981|1250| 500|    30|  2022-01-01|
| 7566| JONES|  MANAGER|7839|04-02-1981|2975|null|    20|  2022-01-05|
| 7654|MARTIN| SALESMAN|7698|21-09-1981|1250|1400|    30|  2022-01-03|
| 7698|   SGR|  MANAGER|7839|05-01-1981|2850|null|    30|  2022-01-04|
| 7782|  RAVI|  MANAGER|7839|06-09-1981|2450|null|    10|  2022-01-02|
| 7788| SCOTT|  ANALYST|7566|19-04-1987|3000|null|    20|  2022-01-02|
| 7839|  KING|PRESIDENT|null|01-11-1981|5000|null|    10|  2022-01-02|
| 7844|TURNER| SALESMAN|7698|09-08-1981|1500|   0|    30|  2022-01-02|
| 7876| ADAMS|    CLERK|7788|23-05-1987|1100|null|    20|  2022-01-03|
| 7900

Write a SQL quesry to find the salary for the emp whose salary is greater than his manager. Lets say 1- joe manager is 3-sam. So comparing this Joe salary is high so that needs to be printed.


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df=spark.read.options(delimiter="|",Header=True).csv("/content/drive/MyDrive/Datasets/bhatth.txt").drop("_c0","_c5")
df.show()
df.createOrReplaceTempView("system")
df=spark.sql("select s1.name from system s1 join system s2 on s1.managerId=s2.id where s1.salary>s2.salary")
df.show()


+---+-----+------+---------+
| id| name|salary|managerId|
+---+-----+------+---------+
|  1|  Joe| 70000|        3|
|  2|Henry| 80000|        4|
|  3|  Sam| 60000|     NULL|
|  4|  Max| 90000|        0|
|  5|Shaen| 70000|        7|
|  6| Mary| 80000|        8|
|  7| Rude| 20000|        4|
|  8| Paul| 72000|        1|
+---+-----+------+---------+

+-----+
| name|
+-----+
|  Joe|
|Shaen|
| Mary|
| Paul|
+-----+




The roundtrip distance should be calculated.using spark or SQL.using the above dataframe.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df=spark.read.options(delimiter=",",Header=True).csv("/content/drive/MyDrive/Datasets/read2.txt")
df.createOrReplaceTempView("xyz")
df=spark.sql("SELECT x1.FROM as from, x1.TO as to, x1.DIST + y1.DIST AS ROUNDTRIP_DIST, 'Shuttle' AS TRANSPORT_MODE FROM xyz x1 JOIN xyz y1 ON x1.FROM = y1.TO AND x1.TO = y1.FROM WHERE x1.FROM < y1.FROM" )
df.show()

+----+---+--------------+--------------+
|from| to|ROUNDTRIP_DIST|TRANSPORT_MODE|
+----+---+--------------+--------------+
| SEA| SF|         600.0|       Shuttle|
| CHI|SEA|        4000.0|       Shuttle|
| LND|SEA|        1000.0|       Shuttle|
+----+---+--------------+--------------+



In [ ]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.getOrCreate()
sc = spark.sparkContext
rdd=sc.textFile("/content/drive/MyDrive/Datasets/SECONDHIGHEST_SALARY.txt")


flat_mapped_rdd = rdd.flatMap(lambda x: x.split(","))
data = flat_mapped_rdd.collect()
for item in data:
    print(item)

flat_mapped_rdd.saveAsTextFile("/content/drive/MyDrive/Datasets/output.csv")




name 			travel_location				 age
ravi			pune
delhi
chennai
noida		32
gautham			delhi
chennai				30
mary			noida
pune				35
thomas			delhi
pune				31
shankar			chennai
noida				30
